In [1]:
from argparse import ArgumentParser
from snewpy import to_snowglobes
from snewpy import run_snowglobes
from snewpy import from_snowglobes
import tarfile
import numpy as np
import os
from astropy.io import ascii
from astropy import units as u
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import gridspec

rng = np.random.default_rng()


mpl.rcParams['figure.figsize']=(6.0,4.0)    #(6.0,4.0)
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['legend.handlelength']=2
mpl.rcParams['legend.fontsize']=16
mpl.rcParams['legend.frameon']=False
mpl.rcParams['axes.labelsize']=18
mpl.rcParams['xtick.labelsize']=16
mpl.rcParams['ytick.labelsize']=16
mpl.rcParams['legend.labelspacing'] = 0.1
mpl.rcParams['xtick.minor.visible'] = True
mpl.rcParams['ytick.minor.visible'] = True
mpl.rcParams['ytick.right'] = True
mpl.rcParams['xtick.top'] = True
mpl.rcParams['ytick.direction'] = 'inout'
mpl.rcParams['xtick.direction'] = 'inout'

home_directory = os.getcwd()
SNOwGLoBES_path = "/Users/eoco/research/github/snowglobes/" #where snowglobes is located


In [2]:
#set distance in kpc
distance=10

#set SNOwGLoBES detector to use
detectors = ["wc100kt30prct"]

#set SNEWPY model type and filename
modeltype = 'OConnor_2015'
modeldir = '/Users/eoco/research/github/NeutrinoEchos/scripts/'
model = 's40_atBH_cutoff'
filename = model+'.dat'

#to_snowglobes.generate_fluence integrates the model over the
#specified time window(s) and generates fluence files for snowglobes

#Option 1 - don't specify tstart and tend, then the whole model is integrated
#tstart = None
#tend = None

#Option 2 - specify single tstart and tend, this makes 1 fluence file integrated over the windown
#tstart = 0.53700724634 *u.s
#tend= 0.53739724633 *u.s

#Option 3 = specify sequence of time intervals, one fluence
#file is made for each elements with dt=tstart[i]-tend[i]
window_tstart = 0.53700724634
window_bins = 39
window_tend = 0.53739724633
tstart = np.linspace(window_tstart,window_tend,window_bins,endpoint=False)*u.s
tend = tstart + (window_tend-window_tstart)/window_bins*u.s
tmid = (tstart+tend)*0.5

#set desired oscillation prescription
transformation = 'AdiabaticMSW_NMO'
#transformation = 'AdiabaticMSW_IMO'

#to_snowglobes creates a tarred file of snowglobes fluences
#this is stored in the model types directory in the snewpy/models
#the full path is returned from to_snowglobes
outfile = modeltype+"_"+model+"_"+transformation

In [3]:
#call to_snowglobes
tables = {}

print("Preparing fluences...")
tarredfile = to_snowglobes.generate_fluence(modeldir, filename, modeltype, transformation, distance, outfile,tstart,tend)
print("Done fluences...")

for detector in detectors:
    print("Running snowglobes with",detector,"...")
    #now run snowglobes, this will loop over all the fluence files in `tarredfile`
    run_snowglobes.go(SNOwGLoBES_path, modeldir, tarredfile, detector_input=detector,verbose=False)
    print("Done snowglobes...")
    
    #now collate results of output of snowglobes
    print("Collating...")
    tables[detector] = from_snowglobes.collate(SNOwGLoBES_path, modeldir, tarredfile, detector_input=detector,skip_plots=True,return_tables=True,verbose=False)

os.chdir(home_directory)
np.save(modeldir+'NeutrinoEcho_'+outfile+'.npy',tables)

Preparing fluences...
[INFO] The `generate_fluence` function has been moved to the `snewpy.snowglobes` module.
Done fluences...
Running snowglobes with wc100kt30prct ...
[INFO] The `go` function has been moved to the `snewpy.snowglobes` module.


Detectors: 100%|██████████| 1/1 [00:06<00:00,  6.53s/it]


Done snowglobes...
Collating...
[INFO] The `collate` function has been moved to the `snewpy.snowglobes` module.


In [4]:
tables = np.load("./NeutrinoEcho_OConnor_2015_s40_atBH_cutoff_AdiabaticMSW_NMO.npy",allow_pickle=True).item()
hyperKfactor=2.2
nevents = {}
for detector in detectors:
    nevents[detector] = np.zeros(len(tmid))
    for i in range(len(tmid)):
        key = "Collated_"+outfile+"_"+str(i)+"_"+detector+"_events_smeared_weighted.dat"
        for j in range(1,len(tables[detector][key]['header'].split())):
            nevents[detector][i] += sum(tables[detector][key]['data'][j])

In [15]:
print("Total Events Hyper-K-like between 0.5370 and 0.5474:",hyperKfactor*sum(nevents["wc100kt30prct"][tmid>(0.5370)*u.s]))
print("Total Events Hyper-K-like between 0.5371 and 0.5474:",hyperKfactor*sum(nevents["wc100kt30prct"][tmid>(0.5371)*u.s]))
print("Total Events Hyper-K-like between 0.5372 and 0.5474:",hyperKfactor*sum(nevents["wc100kt30prct"][tmid>(0.5372)*u.s]))
print("Total Events Hyper-K-like between 0.5373 and 0.5474:",hyperKfactor*sum(nevents["wc100kt30prct"][tmid>(0.5373)*u.s]))
print("Total Events Hyper-K-like between 0.5374 and 0.5474:",hyperKfactor*sum(nevents["wc100kt30prct"][tmid>(0.5374)*u.s]))

Total Events Hyper-K-like between 0.5370 and 0.5474: 18.981841332508107
Total Events Hyper-K-like between 0.5371 and 0.5474: 11.079085194723266
Total Events Hyper-K-like between 0.5372 and 0.5474: 4.914859403930127
Total Events Hyper-K-like between 0.5373 and 0.5474: 1.8086542189297483
Total Events Hyper-K-like between 0.5374 and 0.5474: 0.0
